In [24]:
#import statements
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.constants import G, k_B, h, c, M_sun, L_sun

ALL UNITS IN CGS

In [25]:
#define constants in CGS
G = G.cgs
k_B = k_B.cgs
h = h.cgs
c = c.cgs
M_sun = M_sun.cgs
L_sun = L_sun.cgs

In [26]:
#pull just the value to avoid conflicts with unitless/questionable-unit parameters
G = G.cgs.value
k_B = k_B.cgs.value
h = h.cgs.value
c = c.cgs.value
M_sun = M_sun.cgs.value
L_sun = L_sun.cgs.value

In [43]:

class Accretion_Polynomial:
    
    """
    This class will create objects that store Shakura-Sunyaev accretion disk information on a black hole of a 
    given mass.

    """
    def __init__(self, black_hole_mass, radius, region):
        
        #initialize bulk parameters
        Jstar = 1#boilerplate for now just so code will run
        M = G*black_hole_mass/(c**2)
        a = Jstar/(black_hole_mass*c)
        astar = a/M
        m = M/M_sun
        Delta = 1 #boilerplate for now just so code will run
        Sigma = 1 #boilerplate for now just so code will run
        V = 0.5 #boilerplate for now just so code will run
        L_edd = 3.2e4*m*L_sun #assumes fully ionized hydrogen plasma, but this should suffice for OOM
        Mdot = -2*np.pi*Sigma*np.sqrt(Delta)*V/np.sqrt(1-V**2)
        mdot = Mdot*(c**2)/L_edd
        rstar = radius*(c**2)/(G*M)
        y = (radius/M)**0.5
        A = 1+astar**2*y**-4+2*astar*y**-6
        B = 1+astar*y**-3
        C = 1-3*y**-2+2*astar*y**-3
        D = 1-2*y**-2+astar**2*y**-4
        E = 1+4*astar**2*y**-4-4*astar**2*y**-6+3*astar**4*y**-8
        Q0 = (1+astar*y**-3)/(y*(1-3*y**-2+2*astar*y**-3)**0.5)
        y0 = (radius/M)**0.5
        y1 = 2*np.cos((np.arccos(astar)-np.pi)/3)
        y2 = 2*np.cos((np.arccos(astar)+np.pi)/3)
        y3 = -2*np.cos((np.arccos(astar))/3)
        Q = Q0*(y-y0-1.5*astar*np.log(y/y0)-3*(y1-astar)**2*np.log((y-y1)/(y0-y1))/(y1*(y1-y2)*(y1-y3))) - \
            Q0*(3*(y2-astar)**2/(y*(y2-y1)*(y2-y3))*np.log((y-y2)/(y0-y2))-3*(y3-astar)**2*np.log((y-y3)/(y0-y3))/(y3*(y3-y1)*(y3-y1)))
        
        rho_cgs = 1
        T_K = 1e5
        
        """Use opacity assumptions from Abramowicz 29"""
        kappa_es = 0.34 #cm^2 g^-1
        kappa_ff = 6.4e22*rho_cgs*T_K**(-7/2) #same units as above; rho_cgs is density in g cm^-3, T_K is temp in Kelvin
        if region == 'outer':
            self.kappa = kappa_ff
            self.F = (7e26)*(mdot/m)*(rstar**-3)*(B**-1)*(C**-0.5)*(Q)
            self.Sigma = (4e5)*(alpha**(-0.1)*m**(0.2)*mdot**(0.7))*(rstar**(-3/4))*A**(0.1)*B**(-4/5)*C**(1/2)*D**(-17/20)*E**(-1/20)*Q**(7/10) #Sigma is the surface density
            self.H = (4e2)*(alpha**(-0.1)*m**(18/20)*mdot**(3/20))*rstar**(9/8)*A**(19/20)*B**(-11/10)*C**(1/2)*D**(-23/40)*E**(-19/40)*Q**(3/20)
            self.rhonaught = (4e2)*alpha**(-7/10)*m**(-7/10)*mdot**(3/20)*rstar**(-15/8)*A**(-17/20)*B**(3/10)*D**(-11/40)*E**(17/40)*Q(11/20)
            self.T = (2e8)*(alpha**(-1/5)*m**(-1/5)*mdot**(3/10))*rstar**(-3/4)*A**(-1/10)*B**(-1/5)*D**(-3/20)*E**(1/20)*Q(3/10)
        elif region == 'middle':
            self.kappa = kappa_es
            self.F = (7e26)*(mdot/m)*(rstar**-3)*(B**-1)*(C**-0.5)*(Q)
            self.Sigma = (4e5)*(alpha**(-0.1)*m**(0.2)*mdot**(0.7))*(rstar**(-3/4))*A**(0.1)*B**(-4/5)*C**(1/2)*D**(-17/20)*E**(-1/20)*Q**(7/10) #Sigma is the surface density
            self.H = (4e2)*(alpha**(-0.1)*m**(18/20)*mdot**(3/20))*rstar**(9/8)*A**(19/20)*B**(-11/10)*C**(1/2)*D**(-23/40)*E**(-19/40)*Q**(3/20)
            self.rhonaught = (4e2)*alpha**(-7/10)*m**(-7/10)*mdot**(3/20)*rstar**(-15/8)*A**(-17/20)*B**(3/10)*D**(-11/40)*E**(17/40)*Q(11/20)
            self.T = (2e8)*(alpha**(-1/5)*m**(-1/5)*mdot**(3/10))*rstar**(-3/4)*A**(-1/10)*B**(-1/5)*D**(-3/20)*E**(1/20)*Q(3/10)
        elif region == 'inner':
            self.kappa = kappa_es
            self.F = (7e26)*(mdot/m)*(rstar**-3)*(B**-1)*(C**-0.5)*(Q)
            self.Sigma = (4e5)*(alpha**(-0.1)*m**(0.2)*mdot**(0.7))*(rstar**(-3/4))*A**(0.1)*B**(-4/5)*C**(1/2)*D**(-17/20)*E**(-1/20)*Q**(7/10) #Sigma is the surface density
            self.H = (4e2)*(alpha**(-0.1)*m**(18/20)*mdot**(3/20))*rstar**(9/8)*A**(19/20)*B**(-11/10)*C**(1/2)*D**(-23/40)*E**(-19/40)*Q**(3/20)
            self.rhonaught = (4e2)*alpha**(-7/10)*m**(-7/10)*mdot**(3/20)*rstar**(-15/8)*A**(-17/20)*B**(3/10)*D**(-11/40)*E**(17/40)*Q(11/20)
            self.T = (2e8)*(alpha**(-1/5)*m**(-1/5)*mdot**(3/10))*rstar**(-3/4)*A**(-1/10)*B**(-1/5)*D**(-3/20)*E**(1/20)*Q(3/10)
        else:
            pass
        




In [33]:
#Initialize input arrays
resolution = 1000 #number of steps over the radius
radius = np.arange(resolution)


In [44]:
x = Accretion_Polynomial(1, 1, 'outer')

<ipython-input-43-7bbed9a34620>:31: RuntimeWarning: invalid value encountered in arccos
  y1 = 2*np.cos((np.arccos(astar)-np.pi)/3)
<ipython-input-43-7bbed9a34620>:32: RuntimeWarning: invalid value encountered in arccos
  y2 = 2*np.cos((np.arccos(astar)+np.pi)/3)
<ipython-input-43-7bbed9a34620>:33: RuntimeWarning: invalid value encountered in arccos
  y3 = -2*np.cos((np.arccos(astar))/3)


NameError: name 'alpha' is not defined